# Clean Data for Analysis
Summary needed

In [39]:
import pandas as pd

pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:,.3f}'.format

In [40]:
# From process_data.ipynb, we have the following:
# geo_parcels: parcel boundaries and cities
# digest_full: all digest data where LUC == 101
# sales_full: all sales data LUC == 101

DATA_PATH = 'output/'
geo_parcels = pd.read_csv(DATA_PATH + 'parcels_geo.csv')
digest_full = pd.read_parquet(DATA_PATH + 'digest_full.parquet')
sales_full = pd.read_parquet(DATA_PATH + 'sales_full.parquet')

In [41]:
# Adjust diff for inflation, using CPI-U from the
# BLS: https://data.bls.gov/timeseries/CUUR0000SA0
# Use December CPI for each year as demoninator, and
# 2022 December CPI as numerator
december_2022_cpi = 296.797
december_cpi_year = {
    2010: 219.179,
    2011: 225.672,
    2012: 229.601,
    2013: 233.049,
    2014: 234.812,
    2015: 236.525,
    2016: 241.432,
    2017: 246.524,
    2018: 251.233,
    2019: 256.974,
    2020: 260.474,
    2021: 278.802,
    2022: 296.797
}
inflation_scalar = {x: december_2022_cpi / december_cpi_year[x] for x in december_cpi_year}
print(f"Inflation scalars: {inflation_scalar}")

Inflation scalars: {2010: 1.3541306420779364, 2011: 1.3151698039632742, 2012: 1.2926642305564873, 2013: 1.2735390411458536, 2014: 1.2639771391581351, 2015: 1.25482295740408, 2016: 1.2293192286026708, 2017: 1.2039274066622319, 2018: 1.1813615249589027, 2019: 1.1549689851891634, 2020: 1.1394496187719314, 2021: 1.0645440133141082, 2022: 1.0}


## Parcel Cleaning
- Drop duplicate TAXYR, PARID keys (parcels that contain multiple structures, these are essentially ADUs since we are only looking at single-family); retain only the row with the largest living area square footage (main structure).

Only keep one record per parcel per year (building with greatest SF)

In [42]:
init_len = len(digest_full)
digest_full = digest_full.sort_values(by="Sfla").drop_duplicates(subset=["TAXYR", "PARID"], keep="first")

print(f"Number of dropped duplicates: {init_len - len(digest_full)}")
print(f"Number of total parcels (all years): {len(digest_full)}")

Number of dropped duplicates: 26988
Number of total parcels (all years): 2751045


Drop rows without owner address

In [43]:
# Drop rows without an owner address
init_len = len(digest_full)
digest_full = digest_full[digest_full["Owner Adrstr"] != ""]
print(f"Number of empty addresses dropped: {init_len - len(digest_full)}")

Number of empty addresses dropped: 3539


In [44]:
count_of_records_parcel = pd.DataFrame(digest_full.groupby(by="PARID")["PARID"].count()).rename(
    columns={"PARID": "count_records"}
)
print(f"Total number of unique parcels: {len(count_of_records_parcel)}")

count_missing_a_record = (count_of_records_parcel["count_records"] < 13).sum()
print(f"Nun parcels without a record for every year in period: {count_missing_a_record}")
print("")

tot_parcels_begin = digest_full[digest_full["TAXYR"] == 2010].merge(
    count_of_records_parcel,
    on="PARID",
    how="inner"
)

begin_and_missing = (tot_parcels_begin["count_records"] < 13).sum()
tot_count_missing = tot_parcels_begin["PARID"].count() - begin_and_missing
print(f"Total number of parcels around at start of study period: {len(tot_parcels_begin)}")
print(f"Num of parcels around at start that don't have an entry for every year: {begin_and_missing}")

Total number of unique parcels: 227922
Nun parcels without a record for every year in period: 40564

Total number of parcels around at start of study period: 205697
Num of parcels around at start that don't have an entry for every year: 18339


### Adjust Aprtot for inflation

In [45]:
digest_full["Aprtot_adj"] = digest_full["Aprtot"] * digest_full["TAXYR"].map(inflation_scalar)
digest_full[["TAXYR", "Aprtot", "Aprtot_adj"]].sample(3)

,TAXYR,Aprtot,Aprtot_adj
2455476,2015,"411,000.000","515,732.235"
1646208,2015,"474,100.000","594,911.564"
1662149,2015,"329,400.000","413,338.682"


## Sales Cleaning
- Drop non-arms length transcations (Saleval codes = D, T, G, 4, 4A, 4B, 4C, 4D, 4E, 6F), price under 1K
- Only keep one record when a parcel has been sold twice in one day, with preference for keeping record with Saleval == M. If a parcel had multiple sales in one day, we assume only one of these may have been a market sale and the rest were transfers (manual inspection appears to reveal this).
- Replace sales price for multi-parcel sales with sales price / number of parcels in that transcation. Sales price is total transcation value, which includes many parcels for a multi-parcel sale. 

Saleval code distribution before cleaning

In [46]:
sales_full["Saleval"].value_counts().head(10)

Saleval
0     117330
T      36730
G      23159
5      17287
M      13777
9      13131
3      10205
RE      8124
4       7015
4E      6112
Name: count, dtype: Int64

Drop non-arms length transactions
- Saleval codes: D, T, G, 4, 4B, 4C, 6B, 6F
- Any remaining sales where sales price is under 1K

In [47]:
drop_saleval = ["D", "T", "G", "4", "4A", "4B", "4C", "4D", "4E", "6F"]
init_len = len(sales_full)
sales_full["Saleval"] = sales_full["Saleval"].str.strip()
sales_full = sales_full[~sales_full["Saleval"].isin(drop_saleval)]
sales_full = sales_full[sales_full["SALES PRICE"] > 1000]
print(f"Number of dropped records: {init_len - len(sales_full)}")
print(f"Number of total sales records: {len(sales_full)}")

Number of dropped records: 85636
Number of total sales records: 189717


Examine rows where PARID and Saledt are identical (e.g. potentially duplicate records)

In [48]:
sales_full[sales_full.duplicated(subset=["PARID", "Saledt"])].sort_values(by=["PARID", "Saledt"]).head(5)

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE
98503,2016,06 0310 LL0482,101,17-DEC-2015,"630,000.000","630,000.000",WD,"638,500",9,"CDG HOMES, LLC",HARBERT FRANCETTA F
98510,2016,06 031100040064,101,29-JUL-2015,"603,500.000","603,500.000",SW,"522,400",9,"NATIONAL RESIDENTIAL NOMINEE SERVICES, I",BAYLIN TRACEY
242337,2022,06 031100060237,101,08-FEB-2021,"540,000.000","540,000.000",WD,"557,300",0,PANLILIO LUTGARDO G & VIVIAN L,GOLDNER RONALD E & ARIENE S
126715,2018,06 0312 LL0597,101,28-FEB-2017,"745,000.000","674,900.000",WD,"674,900",P,"HIGHGROVE HOMES, LLC",CARRATO MICHAEL J &
22,2011,06 031200010066,101,07-SEP-2010,"152,200.000","211,500.000",SW,211500,5,"SUNTRUST MORTGAGE, INC.",FEDERAL NATIONAL MORTGAGE ASSOCIATION


In [49]:
sales_full[sales_full["PARID"] == "06 031200020099"]

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE
26787,2012,06 031200020099,101,03-MAY-2011,"192,400.000","85,700.000",DP,85700,5,MATASSINO JOSEPH A JR,PNC BANK N A
26788,2012,06 031200020099,101,03-MAY-2011,"192,400.000","85,700.000",SW,85700,2,PNC BANK N A,FEDERAL HOME LOAN MORTGAGE CORP


Only keep one record where PARID and Saledt. Priority to keep record where Saleval == M.

If a sale happened for the same parcel twice in one day, the assumption that the second sale is non-arms length (e.g. a transfer to some other entity)

In [50]:
init_len = len(sales_full)

# Sort values to keep records where Saleval is M first (in the case of duplicates being dropped)
sales_full["M_flag"] = sales_full["Saleval"].apply(lambda x: True if x == "M" else False)
sales_full = sales_full.sort_values(by=["M_flag"], ascending=False)
sales_full = sales_full.drop_duplicates(subset=["PARID", "Saledt"], keep="first")

print("Count dropped where sale was identical except for Saleval: ", init_len - len(sales_full))

Count dropped where sale was identical except for Saleval:  7679


For multi-parcel sales, the sales price needs to be replaced with sales price divided by number of parcels in the transcation. We consider a record as part of the same multi-parcel sale when Saleval flag == M, Saledt, GRANTEE, and Sales Price are the same.

Drop any records where Saleval == M but there is only one record in that transcation. This indicates it might've been a multi-parcel sale with parcels located outside of Fulton. We do not have complete records for these, and they are an negligible total.

In [51]:
sales_full['sale_size'] = sales_full.groupby(by=["Saledt", "SALES PRICE", "GRANTEE"])["PARID"].transform('count')
sales_full["sales_price"] = sales_full["SALES PRICE"] / sales_full['sale_size']

init_len = len(sales_full)
sales_full = sales_full[~((sales_full["Saleval"] == "M") & (sales_full["sale_size"] == 1))]
print(f"Number of invalid multi-parcel sales dropped: {init_len - len(sales_full)}")

Number of invalid multi-parcel sales dropped: 1346


Verify multi-parcel sale calculation

In [52]:
sales_full[sales_full["Saleval"] != "M"].sample(2)

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE,M_flag,sale_size,sales_price
121784,2017,14 024500010401,101,06-OCT-2016,"42,350.000","42,300.000",SW,"48,100",9,U S BANK NATL ASSN,SECRETARY OF HOUSING & URBAN DEVELOPMENT,False,1,"42,350.000"
111078,2016,17 021500010129,101,30-NOV-2015,"1,236,300.000","1,236,300.000",WD,"2,022,200",9,BREMEN-BOWDON INVESTMENT CO,TED ASSOCIATES INC &,False,1,"1,236,300.000"


In [53]:
sales_full[sales_full["Saleval"] == "M"].sample(2)

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE,M_flag,sale_size,sales_price
253753,2022,13 0132 LL1532,101,23-DEC-2021,"3,200,288.000","102,100.000",LW,"106,500",M,PPIII FM BORROWER LLC,PROMISE HOMES BORROWER I LLC,True,18,"177,793.778"
132464,2018,11 114004080988,101,28-FEB-2017,"400,000.000","542,800.000",LW,"562,700",M,JOHNS CREEK 206 LLC,TPG HOMES AT BELLMOORE LLC,True,2,"200,000.000"


In [54]:
sales_full[
    (sales_full["Saledt"] == "10-MAY-2017")
    & (sales_full["SALES PRICE"] == 265500)
]

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE,M_flag,sale_size,sales_price
137527,2018,14 003700010056,101,10-MAY-2017,"265,500.000","33,000.000",WD,"33,000",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
138501,2018,14 006000030336,101,10-MAY-2017,"265,500.000","43,900.000",WD,"43,900",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
137567,2018,14 003800030772,101,10-MAY-2017,"265,500.000","43,700.000",WD,"43,700",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
137576,2018,14 003800040292,101,10-MAY-2017,"265,500.000","46,000.000",WD,"46,000",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
137586,2018,14 003800050192,101,10-MAY-2017,"265,500.000","34,800.000",WD,"34,800",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"


Saleval codes after cleaning

In [55]:
print(f"Number of total sales records: {len(sales_full)}")
sales_full.groupby("Saleval")["Saleval"].count().sort_values(ascending=False).head(10)

Number of total sales records: 180692


Saleval
0     116433
9      12515
5      11706
3       9586
RE      8034
M       6491
2       4642
W       3515
6       1165
5A      1134
Name: Saleval, dtype: int64

### Adjust sales price, fmv for inflation

In [56]:
sales_full["price_adj"] = sales_full["sales_price"] * (sales_full["TAXYR"] - 1).map(inflation_scalar)
sales_full["fmv_adj"] = sales_full["FAIR MARKET VALUE"] * (sales_full["TAXYR"] - 1).map(inflation_scalar)
sales_full[["TAXYR", "sales_price", "price_adj", "FAIR MARKET VALUE", "fmv_adj"]].sample(3)

,TAXYR,sales_price,price_adj,FAIR MARKET VALUE,fmv_adj
188974,2020,"350,000.000","404,239.145","339,400.000","391,996.474"
270019,2022,"4,250,000.000","4,524,312.057","3,867,100.000","4,116,698.154"
124895,2017,"185,000.000","227,424.057","147,500.000","181,324.586"


## Save

In [57]:
OUTPUT_PATH = 'output/'

digest_full.to_parquet(OUTPUT_PATH + 'digest_full_clean.parquet')
sales_full.to_parquet(OUTPUT_PATH + 'sales_full_clean.parquet')